# AI-Powered Personalized Study Planner

Welcome to our AI Study Planner (Consolidated Version). This notebook allows to:
1.  **Train a Machine Learning Model** on study data.
2.  **Interactive UI**: Add subjects and set goals using widgets.
3.  **Generate a Weekly Schedule** tailored to your daily availability.
4.  **Sync with Google Calendar** (Directly from this notebook).

In [5]:
# 1. Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import ipywidgets as widgets
from IPython.display import display, clear_output
import datetime
import os.path

# Google Calendar Imports
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Configuration
DATA_PATH = 'data/study_data.csv'
SCOPES = ['https://www.googleapis.com/auth/calendar']

print("Libraries Imported Successfully!")

Libraries Imported Successfully!


## 2. Train the Model

In [6]:
def train_model():
    try:
        if not os.path.exists(DATA_PATH):
             # Create dummy data if missing
            print("Data file not found. Creating dummy data...")
            os.makedirs('data', exist_ok=True)
            df_dummy = pd.DataFrame({
                'Difficulty_Level': np.random.randint(1, 6, 50),
                'Marks_Obtained': np.random.randint(50, 100, 50),
                'Hours_Studied': np.random.randint(1, 10, 50)
            })
            df_dummy.to_csv(DATA_PATH, index=False)
            
        df = pd.read_csv(DATA_PATH)
        X = df[['Difficulty_Level', 'Marks_Obtained']]
        y = df['Hours_Studied']
        
        model = LinearRegression()
        model.fit(X, y)
        
        print(f"Model Trained Successfully!")
        return model
    except Exception as e:
        print(f"Error training model: {e}")
        return None

model = train_model()

Model Trained Successfully!


## 3. Google Calendar Authentication Logic

In [7]:
def authenticate_google_calendar():
    creds = None
    # The file token.json stores the user's access and refresh tokens.
    if os.path.exists('token.json'):
        try:
            creds = Credentials.from_authorized_user_file('token.json', SCOPES)
        except ValueError:
            print("Corrupt 'token.json' found. Will re-authenticate.")
            try:
                 os.remove('token.json')
            except OSError:
                 pass
            creds = None
    
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            try:
                creds.refresh(Request())
            except Exception:
                print("Token expired and refresh failed. Please re-authenticate.")
                creds = None
        
        if not creds:
            if not os.path.exists('credentials.json'):
                print("ERROR: 'credentials.json' not found. Please place it in the project root.")
                return None
            
            try:
                flow = InstalledAppFlow.from_client_secrets_file(
                    'credentials.json', SCOPES)
                # Run local server for auth
                creds = flow.run_local_server(port=0)
            except Exception as e:
                print(f"Authentication Error: {e}")
                return None
        
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    
    return build('calendar', 'v3', credentials=creds)

def create_calendar_event(service, summary, start_time, end_time):
    event = {
        'summary': summary,
        'start': {
            'dateTime': start_time.isoformat(),
            'timeZone': 'UTC',
        },
        'end': {
            'dateTime': end_time.isoformat(),
            'timeZone': 'UTC',
        },
    }
    event = service.events().insert(calendarId='primary', body=event).execute()
    print(f'Event created: {event.get("htmlLink")}')

## 4. Interactive Planner Application

In [ ]:
# State
subjects_list = []
generated_schedule = []

# --- WIDGETS ---

# 1. Subject Navigation
txt_subject = widgets.Text(description="Subject:", placeholder="e.g. AI")
slider_marks = widgets.IntSlider(value=80, min=0, max=100, step=1, description="Goal Marks:")
toggle_difficulty = widgets.ToggleButtons(
    options=[('Easy', 1), ('Medium', 3), ('Hard', 5)],
    description='Difficulty:',
)

# 2. Weekly Schedule Settings (Per Day)
days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_widgets = {}

# Create widgets for each day
schedule_rows = []
for day in days_of_week:
    # Start Time (0-23 hours)
    start_drop = widgets.Dropdown(
        options=[(f"{h}:00", h) for h in range(7, 24)] + [(f"{h}:00", h) for h in range(0, 7)],
        value=10,
        description=f'{day} Start:',
        style={'description_width': 'initial'}
    )
    # Duration Slider
    duration_slider = widgets.IntSlider(
        value=4, min=0, max=12, step=1, 
        description='Hours:',
        style={'description_width': 'initial'}
    )
    
    day_widgets[day] = {'start': start_drop, 'duration': duration_slider}
    schedule_rows.append(widgets.HBox([start_drop, duration_slider]))

accordion_schedule = widgets.Accordion(children=[widgets.VBox(schedule_rows)])
accordion_schedule.set_title(0, 'Configure Weekly Availability (Click to Expand)')

# 3. Actions
btn_add = widgets.Button(description="Add Subject", button_style='info')
btn_generate = widgets.Button(description="Generate Plan", button_style='primary')
btn_sync = widgets.Button(description="Sync to Calendar", button_style='success')
btn_clear = widgets.Button(description="Clear Subjects", button_style='warning')
output_area = widgets.Output()

# --- LOGIC ---

def refresh_ui(message=None):
    with output_area:
        clear_output()
        if message:
            print(f"[{message}]")
            print("-" * 30)
        
        # Show Subjects
        print(f"Current Subjects ({len(subjects_list)}):")
        for i, s in enumerate(subjects_list):
            print(f"{i+1}. {s['name']} | Goal: {s['marks']} | Diff: {s['difficulty']}")
        
        # Show Schedule if exists
        if generated_schedule:
            print("\n--- Generated Weekly Plan ---")
            for item in generated_schedule:
                # item => (summary, start_dt, end_dt)
                start_str = item[1].strftime("%A %H:%M") # Show DayName Time
                print(f"[ ] {start_str} : {item[0]}")

def on_add_click(b):
    if txt_subject.value:
        subjects_list.append({
            'name': txt_subject.value,
            'marks': slider_marks.value,
            'difficulty': toggle_difficulty.value
        })
        txt_subject.value = ""
        refresh_ui("Subject Added")
    else:
        refresh_ui("Error: Enter subject name")

def on_clear_click(b):
    global subjects_list, generated_schedule
    subjects_list = []
    generated_schedule = []
    refresh_ui("Cleared Data")

def on_generate_click(b):
    global generated_schedule
    if not model:
        refresh_ui("Error: Model not trained")
        return
    if not subjects_list:
        refresh_ui("Error: No subjects to plan")
        return

    # 1. Predict Hours for each subject
    plan_queue = []
    for s in subjects_list:
        pred = model.predict([[s['difficulty'], s['marks']]])[0]
        hours = max(1, round(pred, 1))
        # Store as many 1-hour blocks as needed
        for _ in range(int(hours)):
            plan_queue.append(s['name'])
    
    # 2. Schedule using Daily Configuration
    generated_schedule = []
    start_date = datetime.datetime.now()
    # Try scheduling for next 14 days to be safe
    
    queue_idx = 0
    total_blocks = len(plan_queue)
    
    for day_offset in range(1, 15): # Start tomorrow
        if queue_idx >= total_blocks:
            break
            
        current_day_date = start_date + datetime.timedelta(days=day_offset)
        day_name = current_day_date.strftime("%A")
        
        # Retrieves settings for this specific day
        day_config = day_widgets[day_name]
        start_hour = day_config['start'].value
        max_hours = day_config['duration'].value
        
        # Schedule blocks for this day
        for h in range(max_hours):
            if queue_idx >= total_blocks:
                break
            
            subject_name = plan_queue[queue_idx]
            # Create Time Slot
            slot_start = current_day_date.replace(hour=start_hour + h, minute=0, second=0, microsecond=0)
            slot_end = slot_start + datetime.timedelta(hours=1)
            
            generated_schedule.append((f"Study {subject_name}", slot_start, slot_end))
            queue_idx += 1
    
    if queue_idx < total_blocks:
        refresh_ui(f"Warning: Could not fit all tasks in 2 weeks! Scheduled {queue_idx}/{total_blocks} hours.")
    else:
        refresh_ui("Plan Generated successfully!")

def on_sync_click(b):
    if not generated_schedule:
        refresh_ui("Error: No plan to sync")
        return
        
    refresh_ui("Authenticating with Google...")
    service = authenticate_google_calendar()
    if not service:
        refresh_ui("Authentication Failed")
        return
    
    count = 0
    for summary, start, end in generated_schedule:
        try:
            create_calendar_event(service, summary, start, end)
            count += 1
        except Exception as e:
            print(f"Sync Error: {e}")
    
    refresh_ui(f"Synced {count} events!")

# --- BINDINGS ---
btn_add.on_click(on_add_click)
btn_clear.on_click(on_clear_click)
btn_generate.on_click(on_generate_click)
btn_sync.on_click(on_sync_click)

# --- LAYOUT ---
ui_container = widgets.VBox([
    widgets.HTML("<h3>AI Study Planner</h3>"),
    widgets.HBox([txt_subject, btn_add, btn_clear]),
    widgets.HBox([slider_marks, toggle_difficulty]),
    widgets.HTML("<hr><h4>Available Study Hours</h4>"),
    accordion_schedule,
    widgets.HTML("<br>"),
    widgets.HBox([btn_generate, btn_sync]),
    output_area
])

print("Initializing UI...")
display(ui_container)
refresh_ui("Ready")

Initializing UI...
